# Mater Academy Data Analytics Project
created by: Eddy Varela

### In this project, we are trying to keep track of the students who improve in every quarter in order to reward their efforts to improve their grades.

Here we calculate the improvement per quarter

In [3]:
import csv
import pandas as pd

def make_excel(data_list,output):
    with open(str(output)+'.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, dialect='excel')

        for i in data_list:
            writer.writerow(i)
            
    
def calc_difference(first,second):
    difference = 0
    if first == "" and second == "":
        return difference
    elif first == " ":
        first = second
        return difference

    elif second == " ":
        second = first
        return difference
    else:
        
        if(first == "F"):
            first = "E"
        if(second == "F" ):
            second ="E"
        difference = (ord(first) - ord(second))

    return difference


def get_firstID(filepath):
    with open(filepath) as file:
        reader = csv.reader(file)
        reader.__next__()
        num = reader.__next__()[0]
    return num

def calculate_improvement(filepath,output):
    print(str(output)+ ":")
    l = []
    with open(filepath) as file:
        reader = csv.reader(file)
        reader.__next__() #Skip first row
        points = 0
        IDnum = 0
        pre= []
        IDnum = get_firstID(filepath)

        for row in reader:
    
            firstNineWeeks = row[5]
            secondNineWeeks = row[6]
            checkID = row[0]
            name = row[3] + " "+ row[4]
            grade = row[2]

            if (IDnum != checkID):
                l.append([pre[0],pre[2], pre[3]+ " " +pre[4] , points])
                points = 0
                IDnum = checkID
            
            points += calc_difference(firstNineWeeks, secondNineWeeks)
            pre = row
    make_excel(l,output)


Here we are going to look at students who improve quarter over quarter

In [123]:
def continuous_improvement(q1_csv, q2_csv, output):
#     calculate_improvement(q1_csv, output)
#     calculate_improvement(q2_csv, output+"2")
    clean_files(q1_csv, q2_csv, "./"+output+".csv")
    
    
def clean_files(f1, f2, out):
    df_1 = pd.read_csv(f1, names = ["ID", "Grade", "Name", "Points"])
    df_2 = pd.read_csv(f2, names = ["ID", "Grade", "Name", "Points"])
    
    df_1 = df_1.merge(df_2, left_on = "ID", right_on="ID")
    df_1 = df_1.drop(df_1.columns[4:6], axis=1)

    print(df_1)
    
    r_df = pd.DataFrame()
    for ind in df_1.index:
        if (df_1["Points_x"][ind]>0 and df_1["Points_y"][ind]>0):
            r_df = r_df.append(df_1.iloc[ind])
    
    
    r_df.set_index("ID").to_csv(out)


In [124]:
continuous_improvement('./6012_q1.csv','./6012_q2-q3.csv', "6012_improve")
continuous_improvement('./7014_q1.csv','./7014_q2-q3.csv', "7014_improve")

# continuous_improvement('./data/7014_1.csv', './data/7014.csv', "7014_continuous_improvement")
# continuous_improvement('./data/6012_1.csv', './data/6012.csv', "6012_continuous_improvement")

          ID  Grade_x                 Name_x  Points_x  Points_y
0      89326        8        ADRIANA CABRERA        -1         0
1     143026        7         JAMARIYA MARCH         0         3
2     209643        8           LISBETH DIAZ         1         0
3     212905        8        JOSHUA RADCLIFF         5         0
4     230270        8          MICHAEL PEREZ        -1         7
5     246656        7      CHRISTINA MEDEROS        -4         2
6     248013        6       ANTHONY CASTILLO        -1         2
7     255576        8            ANGEL NICOT         0        -1
8     256714        8       CHRISTOPHE PEREZ         1        -2
9     257227        8      GABRIEL RODRIGUEZ         0         0
10    258136        8         NATHAN ZELEDON         0         0
11    260006        8           LILIANA BOZA         2         0
12    260455        7           ANGEL LAREDO        -3         0
13    268657        8         PAMELA SEVILLA         3         1
14    274707        8    